# Advanced Querying Mongo

Importing libraries and setting up connection

In [2]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [3]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [5]:
client.list_database_names()

['admin', 'config', 'ironhack', 'local']

In [6]:
db = client.get_database("ironhack")

In [7]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [11]:
collection = db.get_collection("companies")


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [39]:
collection.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [24]:
proj = {"_id": 0,"name": 1}
list(collection.find({"name":'Babelgum'},proj))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [33]:
proj = {"_id": 0,"name": 1}
list(collection.find({"number_of_employees":{"$gt": 5000}},proj).limit(20).sort("number_of_employees",1))

[{'name': 'Nintendo'},
 {'name': 'Hexaware Technologies'},
 {'name': 'Facebook'},
 {'name': 'OpenText'},
 {'name': 'CPM Braxis'},
 {'name': 'LSI'},
 {'name': 'Microchip Technologies'},
 {'name': 'Mediaset'},
 {'name': 'Mindray Medical International'},
 {'name': 'Tata Communications'},
 {'name': 'Baidu'},
 {'name': 'Dentsu'},
 {'name': 'Atmel'},
 {'name': 'Acxiom'},
 {'name': 'Acxiom'},
 {'name': 'SRA International'},
 {'name': "Moody's"},
 {'name': 'ManTech'},
 {'name': 'Adobe Systems'},
 {'name': 'Broadcom'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [37]:
proj = {"_id": 0,"name": 1,"founded_year":1}
condicion1 = {"founded_year": {"$gte": 2000}}
condicion2 = {"founded_year": {"$lte": 2005}}
list(collection.find({"$and": [condicion1,condicion2]},proj).limit(20))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [41]:
proj = {"_id": 0,"name": 1,"ipo":1}
condicion1 = {"ipo.valuation_amount": {"$gte": 100000000}}
condicion2 = {"founded_year": {"$lte": 2010}}
list(collection.find({"$and": [condicion1,condicion2]},proj).limit(10))

[{'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Brightcove',
  'ipo': {'valuation_amount': 290000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 2,
   'pub_day': 17,
   'stock_symbol': 'NASDAQ:BCOV'}},
 {'name': 'Amazon',
  'ipo': {'valu

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [42]:
proj = {"_id": 0,"name": 1}
condicion1 = {"number_of_employees":{"$lt": 1000}}
condicion2 = {"founded_year": {"$lt": 2005}}
list(collection.find({"$and": [condicion1,condicion2]},proj).limit(10).sort("number_of_employees",1))

[{'name': 'Fox Interactive Media'},
 {'name': 'Ticketmaster'},
 {'name': 'Eurekster'},
 {'name': 'MindTouch'},
 {'name': 'Skype'},
 {'name': 'Monster'},
 {'name': 'EditGrid'},
 {'name': 'Simpy'},
 {'name': 'Compete'},
 {'name': 'stylediary'}]

### 6. All the companies that don't include the `partners` field.

In [45]:
proj = {"_id": 0,"name": 1}
list(collection.find({"partners":{"$exists":False}},proj))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [113]:
proj = {"_id": 0,"name": 1,"category_code":1}
list(collection.find({"category_code":{ "$type": 10}},proj).limit(10))

[{'name': 'Collective', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None},
 {'name': 'Drigg', 'category_code': None},
 {'name': 'SpaceTime', 'category_code': None},
 {'name': 'MMDAYS', 'category_code': None},
 {'name': 'Touch Clarity', 'category_code': None},
 {'name': 'Inside Group', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [60]:
proj = {"_id": 0,"name": 1,"number_of_employees":1}
condicion1 = {"number_of_employees":{"$gte": 100}}
condicion2 = {"number_of_employees":{"$lt": 1000}}
list(collection.find({"$and": [condicion1,condicion2]},proj).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [103]:
proj = {"_id": 0,"name": 1}
list(collection.find({"ipo":{"$exists":True}},proj).limit(10).sort("ipo.valuation_amount",-1))

[{'name': 'GREE'},
 {'name': 'Facebook'},
 {'name': 'Amazon'},
 {'name': 'Twitter'},
 {'name': 'Groupon'},
 {'name': 'Tencent'},
 {'name': 'Western Digital'},
 {'name': 'LinkedIn'},
 {'name': 'BMC Software'},
 {'name': 'Rackspace'}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [79]:
proj = {"_id": 0,"name": 1}
list(collection.find({"number_of_employees":{"$exists":True}},proj).sort("number_of_employees",1).limit(10))

[{'name': 'Omnidrive'},
 {'name': 'Lala'},
 {'name': 'Gizmoz'},
 {'name': 'Flektor'},
 {'name': 'StumbleUpon'},
 {'name': 'CBS'},
 {'name': 'Viacom'},
 {'name': 'Postini'},
 {'name': 'Slacker'},
 {'name': 'Helio'}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [71]:
proj = {"_id": 0,"name": 1}
list(collection.find({"founded_month":{"$gte": 6}},proj).limit(10))

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Postini'},
 {'name': 'Digg'},
 {'name': 'Omnidrive'},
 {'name': 'Geni'},
 {'name': 'Fox Interactive Media'},
 {'name': 'Joost'},
 {'name': 'Plaxo'},
 {'name': 'Powerset'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [78]:
proj = {"_id": 0,"name": 1}
condicion1 = {"founded_year":{"$lt": 2000}}
condicion2 = {"acquisition.price_amount":{"$gt": 10000}}
list(collection.find({"$and": [condicion1,condicion2]},proj).limit(10))

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'Cyworld'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'},
 {'name': 'Neopets'},
 {'name': 'Sun Microsystems'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [77]:
proj = {"_id": 0,"name": 1,"acquisition":1}
list(collection.find({"acquisition.acquired_year":{"$gt": 2010}},proj).limit(5).sort("acquisition.price_amount",1))

[{'name': 'Geni',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2012/11/28/all-in-the-family-myheritage-buys-former-yammer-stablemate-geni-com-raises-25m/',
   'source_description': 'MyHeritage acquires Geni and $25M to build family tree of the whole world',
   'acquired_year': 2012,
   'acquired_month': 11,
   'acquired_day': 28,
   'acquiring_company': {'name': 'MyHeritage', 'permalink': 'myheritage'}}},
 {'name': 'Mashery',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://allthingsd.com/20130417/intel-acquires-api-manager-mashery/',
   'source_description': 'Intel Acquires API Manager Mashery',
   'acquired_year': 2013,
   'acquired_month': 4,
   'acquired_day': 17,
   'acquiring_company': {'name': 'Intel', 'permalink': 'intel'}}},
 {'name': 'delicious',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [87]:
proj = {"_id": 0,"name": 1,"founded_year":1}
list(collection.find({"founded_year":{"$exists": True}},proj).limit(10).sort("founded_year",-1))

[{'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [90]:
proj = {"_id": 0,"name": 1,"founded_day":1}
list(collection.find({"founded_day":{"$lte": 7}},proj).limit(10).sort("acquisition.price_amount",-1))

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'Vitrue', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [92]:
proj = {"_id": 0,"name": 1,"category_code":1}
condicion1 = {"category_code":"web"}
condicion2 = {"number_of_employees":{"$gt": 4000}}
list(collection.find({"$and": [condicion1,condicion2]},proj).sort("number_of_employees",1).limit(10))

[{'name': 'Expedia', 'category_code': 'web'},
 {'name': 'AOL', 'category_code': 'web'},
 {'name': 'Webkinz', 'category_code': 'web'},
 {'name': 'Rakuten', 'category_code': 'web'},
 {'name': 'Los Angeles Times Media Group', 'category_code': 'web'},
 {'name': 'Groupon', 'category_code': 'web'},
 {'name': 'Yahoo!', 'category_code': 'web'},
 {'name': 'eBay', 'category_code': 'web'},
 {'name': 'Experian', 'category_code': 'web'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [104]:
proj = {"_id": 0,"name": 1}
condicion1 = {"acquisition.price_amount":{"$gt": 10000000}}
condicion2 = {"acquisition.price_currency_code":"EUR"}
list(collection.find({"$and":[condicion1,condicion2]},proj).limit(10))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [99]:
proj = {"_id": 0,"name": 1,"acquisition.acquired_month":1}
list(collection.find({"acquisition.acquired_month":{"$lte": 3}},proj).limit(10))

[{'name': 'Kyte', 'acquisition': {'acquired_month': 1}},
 {'name': 'NetRatings', 'acquisition': {'acquired_month': 2}},
 {'name': 'blogTV', 'acquisition': {'acquired_month': 3}},
 {'name': 'Livestream', 'acquisition': {'acquired_month': 1}},
 {'name': 'iContact', 'acquisition': {'acquired_month': 2}},
 {'name': 'Coghead', 'acquisition': {'acquired_month': 2}},
 {'name': 'Dailymotion', 'acquisition': {'acquired_month': 2}},
 {'name': 'Netvibes', 'acquisition': {'acquired_month': 2}},
 {'name': 'Flickr', 'acquisition': {'acquired_month': 3}},
 {'name': 'BabyCenter', 'acquisition': {'acquired_month': 3}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [109]:
proj = {"_id": 0,"name": 1}
condicion1 = {"founded_year":{"$gt": 2000}}
condicion2 = {"founded_year":{"$lt": 2010}}
condicion3 = {"acquisition.acquired_year":{"$gt":2011}}
list(collection.find({"$and":[condicion1,condicion2,condicion3]},proj).limit(10))

[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'blogTV'},
 {'name': 'Revision3'},
 {'name': 'iContact'},
 {'name': 'Mashery'},
 {'name': 'Dailymotion'},
 {'name': 'Netvibes'},
 {'name': 'KickApps'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [112]:
proj = {"_id": 0,"name": 1,"deadpooled_year":1}
list(collection.find({"deadpooled_year":{"$gt": 3}},proj).limit(10))

[{'name': 'Omnidrive', 'deadpooled_year': 2008},
 {'name': 'Babelgum', 'deadpooled_year': 2013},
 {'name': 'Sparter', 'deadpooled_year': 2008},
 {'name': 'Thoof', 'deadpooled_year': 2013},
 {'name': 'Mercora', 'deadpooled_year': 2008},
 {'name': 'Wesabe', 'deadpooled_year': 2010},
 {'name': 'Jangl SMS', 'deadpooled_year': 2008},
 {'name': 'Stickam', 'deadpooled_year': 2013},
 {'name': 'AllPeers', 'deadpooled_year': 2008},
 {'name': 'EQO', 'deadpooled_year': 2012}]